# <h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import Requirements" data-toc-modified-id="Import-Requirements-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Requirements</a></span></li><li><span><a href="#Prepare Training Data" data-toc-modified-id="Prepare-Training-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare Training Data</a></span><ul class="toc-item"></ul></li><li><span><a href="#Model Training" data-toc-modified-id="Model Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model Training</a></span></li><li><span><a href="#Model Saving" data-toc-modified-id="Model Saving-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model Saving</a></span><ul class="toc-item"></ul></li><li><span><a href="#Validation and Results" data-toc-modified-id="Validation and Results-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Validation and Results</a></span><ul class="toc-item"></ul></div>

<a id='Import Requirements'></a>

# Import Requirements

In [ ]:
# Make sure to 

In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import string
import matplotlib.pyplot as plt

In [2]:
import sklearn
sklearn.__version__

'0.21.1'

<a id='Prepare Training Data'></a>

# Prepare Training Data

Input data for training consists of both historical data and CICD data( Production run data for which manual agent validation has been done for the ML prediction)

In [3]:
def preprocess_text(message):

    #stopwords
    #new_stopwords=['hi','hello','team','thanks','hey','regards','please','jira','com','ext','image','capture','best','customer','uber','png','mailto']
    stpwrd = nltk.corpus.stopwords.words('english')
    #stpwrd.extend(new_stopwords)
    # 1. Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
    message=message.lower()
    #removing the numerical values and working only with text values
    message = re.sub('[^a-zA-Z]', " ", message)
    #lowering and removing punctuation
    message = re.sub(r'[^\w\s]',' ',message)
    #removing the stopwords
    message = ' '.join([word for word in message.split() if word not in stpwrd and len(word)>1])
    #lemmatizing the text
    message =  " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(message) if w not in string.punctuation])
    #removing hyperlinks
    message = re.sub(r'http\S+', ' ', message)
    #removing special characters
    message=message.replace('<','').replace('>','').replace('*','').replace('[','').replace(']','').replace('|','')
    
    return message

In [4]:
#data_cicd=pd.read_csv('../data/retrain_data/2022-12-13 - INDIRECTTX-1926 - Walgreens_after_tagging.csv' , usecols = ['item_name','subsection_name','description','establishment_type','CAT NAME,CAT_TEMP','Integers'])
#data_cicd['target_new']=data_cicd['CAT NAME,CAT_TEMP'] + ":" + data_cicd['Integers']
#data_cicd.drop(['CAT NAME,CAT_TEMP', 'Integers'],inplace=True,axis=1)
#data_cicd['combined_text'] = final_data[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)


In [29]:
#del data_cicd, cicd_categories_beforemapping, cicd_categories_aftermapping, mapping_data, master_mapping_data, set_cicd_categories_aftermapping, set_master_mapping_data_categories, dif_categories, common_categories
data_his=pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/historical_data_24_11_22.csv')

In [30]:
data_his.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396190 entries, 0 to 396189
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Unnamed: 0          396190 non-null  int64 
 1   UniqueUUID          396190 non-null  object
 2   Item                396187 non-null  object
 3   Description         271012 non-null  object
 4   establishment_type  396190 non-null  object
 5   target_new          396190 non-null  object
dtypes: int64(1), object(5)
memory usage: 18.1+ MB


In [31]:
data_cicd=pd.read_csv('../data/retrain_data/TaxML-CICD - Prod_Data_latest.csv',index_col=0)
#/Users/mdevar2/Downloads/data discripency/TaxML-CICD - Prod_Data (4).csv

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
#data_cicd.shape

In [33]:
data_cicd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 583360 entries, 12/08/22 03:53 to 15/02/23 23:36
Data columns (total 14 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   Date (BOT Sent Details to COE Team)  580535 non-null  object
 1   UniqueUUID                           583360 non-null  object
 2   store_uuid                           583360 non-null  object
 3   item_uuid                            583360 non-null  object
 4   Item                                 583358 non-null  object
 5   Description                          341692 non-null  object
 6   establishment_type                   583360 non-null  object
 7   CAT Name                             583360 non-null  object
 8   Integer                              583360 non-null  object
 9   Confidence Score                     583360 non-null  object
 10  Agent Corrected CAT Name             583360 non-null  object
 11  Agent Corr

In [34]:
#data_cicd.drop('Unnamed: 0')

In [35]:
#data_cicd.rename(columns = {'Agent Corrected CAT NAME':'Agent_Corrected_CAT_NAME'}

In [36]:
data_cicd['Agent Corrected CAT Name'].nunique()

182

In [9]:
data_cicd = data_cicd.dropna(axis=0, subset=['Agent Corrected CAT Name'])
data_cicd = data_cicd.drop(data_cicd[data_cicd['Agent Corrected CAT Name']=='#REF!'].index)
data_cicd = data_cicd.drop(data_cicd[data_cicd['Agent Corrected CAT Name']=='106,107,1'].index)
data_cicd = data_cicd.drop(data_cicd[data_cicd['Agent Corrected CAT Name']=='Bottled Tea Drink'].index)
data_cicd = data_cicd.drop(data_cicd[data_cicd['Agent Corrected CAT Name']=='Bottled tea drink'].index)
data_cicd = data_cicd.drop(data_cicd[data_cicd['Agent Corrected CAT Name']=='Air Freshener'].index)

In [25]:
data_cicd['Agent Corrected Integer'].nunique()

182

In [38]:
master_mapping_data = pd.read_csv("../data/MASTER_Tax_Category_Mapping.csv")
set_cicd_categories_aftermapping = set(data_cicd['Agent Corrected CAT Name'].unique())
set_master_mapping_data_categories = set(master_mapping_data['CAT_Name'].unique())
dif_categories = list(set_cicd_categories_aftermapping.difference(set_master_mapping_data_categories))
len(dif_categories)

0

In [39]:
master_mapping_data = pd.read_csv("../data/MASTER_Tax_Category_Mapping.csv")
set_cicd_categories_aftermapping = set(data_cicd['CAT Name'].unique())
set_master_mapping_data_categories = set(master_mapping_data['CAT_Name'].unique())
dif_categories = list(set_cicd_categories_aftermapping.difference(set_master_mapping_data_categories))
len(dif_categories)

0

In [28]:
dif_categories

[nan,
 'CAT_SANDWICH,TEMP_HEATED',
 'CAT_LIQUOR,TEMP_HEATED',
 'Gancia, 750mL Prosecco (11.5% ABV)',
 'August 29,2022']

In [11]:
#cicd_categories_beforemapping=data_cicd['Agent Corrected CAT Name'].unique()
#cicd_categories_beforemapping=pd.Series(cicd_categories_beforemapping)
#cicd_categories_beforemapping.to_csv('cicd_categories_beforemapping.csv')

In [12]:
mapping_data=pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/retrain_data/mapping_latest.csv')
cat_dict = dict(zip(mapping_data['Agent CAT Name'], mapping_data['Updated CAT Name']))
data_cicd["Agent Corrected CAT Name"] = data_cicd["Agent Corrected CAT Name"].replace(cat_dict)

In [29]:
mapping_data=pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/retrain_data/mapping_latest.csv')
cat_dict = dict(zip(mapping_data['Agent CAT Name'], mapping_data['Updated CAT Name']))
data_cicd["CAT Name"] = data_cicd["CAT Name"].replace(cat_dict)

In [13]:
mapping_data_1=pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/MASTER_Tax_Category_Mapping_latest1.csv')
mapping1 = dict(mapping_data_1[['Updated CAT Name', 'Updated Integer']].values)
data_cicd['Agent Corrected Integer'] = data_cicd['Agent Corrected CAT Name'].map(mapping1)

In [34]:
mapping_data_1=pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/MASTER_Tax_Category_Mapping_latest1.csv')
mapping1 = dict(mapping_data_1[['Updated CAT Name', 'Updated Integer']].values)
data_cicd['Integer'] = data_cicd['CAT Name'].map(mapping1)

In [79]:
data_cicd1=data_cicd[data_cicd['Agent Corrected CAT Name']!=data_cicd['CAT Name']]

In [80]:
#data_cicd1=data_cicd[data_cicd['Agent Corrected Integer']!=data_cicd['Integer']]

In [81]:
data_cicd1['CAT NAME_ ValidationScore [0-100]'].value_counts()

0      11477
100        3
Name: CAT NAME_ ValidationScore [0-100], dtype: int64

In [82]:
data_cicd1['CAT NAME_ ValidationScore [0-100]']=0

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [83]:
data_cicd2=data_cicd[data_cicd['Agent Corrected CAT Name']==data_cicd['CAT Name']]

In [87]:
data_cicd2['CAT NAME_ ValidationScore [0-100]'].value_counts()

100    571891
Name: CAT NAME_ ValidationScore [0-100], dtype: int64

In [86]:
data_cicd2['CAT NAME_ ValidationScore [0-100]']=100

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [88]:
data_cicd_final=pd.concat([data_cicd1,data_cicd2],ignore_index=True)

In [89]:
data_cicd_final1=data_cicd_final[data_cicd_final['Agent Corrected Integer']!=data_cicd_final['Integer']]

In [90]:
data_cicd_final1['Integer_ValidationScore[0-100]'].value_counts()

0      11404
100       76
Name: Integer_ValidationScore[0-100], dtype: int64

In [91]:
data_cicd_final1['Integer_ValidationScore[0-100]']=0

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [92]:
data_cicd_final2=data_cicd_final[data_cicd_final['Agent Corrected Integer']==data_cicd_final['Integer']]

In [93]:
data_cicd_final2['Integer_ValidationScore[0-100]'].value_counts()

100    543443
0       28448
Name: Integer_ValidationScore[0-100], dtype: int64

In [94]:

data_cicd_final2['Integer_ValidationScore[0-100]']=100

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [95]:
data_cicd_final3=pd.concat([data_cicd_final1,data_cicd_final2],ignore_index=True)

In [96]:
#data_cicd_final.sort_values(by='CICD Run Date',ascending=False)
data_cicd_final3=data_cicd_final.sort_values(by=['CICD Run Date', 'Date (BOT Sent Details to COE Team)'],ascending=False)

In [97]:
data_cicd_final3.head()

,CICD Run Date,Date (BOT Sent Details to COE Team),UniqueUUID,store_uuid,item_uuid,Item,Description,establishment_type,CAT Name,Integer,Confidence Score,Agent Corrected CAT Name,Agent Corrected Integer,CAT NAME_ ValidationScore [0-100],Integer_ValidationScore[0-100]
304883,31-10-2022 11:31 PM,19-10-2022 5:03 AM,d6ca3c6a-f434-5b73-b4bb-a3d9b2c75429:d25071ca-...,d6ca3c6a-f434-5b73-b4bb-a3d9b2c75429,d25071ca-2296-4399-99ba-d9070239ced9,Jumbo Size Cupcake (12 pcs),Jumbo size by the dozen.,GROCERY,CAT_BAKERY_ITEM_GROCERY_STORE,558,0.22,CAT_BAKERY_ITEM_GROCERY_STORE,558,100,100
304884,31-10-2022 11:31 PM,19-10-2022 5:03 AM,d6ca3c6a-f434-5b73-b4bb-a3d9b2c75429:f2c13e24-...,d6ca3c6a-f434-5b73-b4bb-a3d9b2c75429,f2c13e24-3aa0-43ae-a23e-7396533745a7,Cinnamon Rolls,NaN,GROCERY,CAT_BAKERY_ITEM,562,0.3,CAT_BAKERY_ITEM,562,100,100
304885,31-10-2022 11:31 PM,19-10-2022 5:03 AM,d6ca3c6a-f434-5b73-b4bb-a3d9b2c75429:b30738ef-...,d6ca3c6a-f434-5b73-b4bb-a3d9b2c75429,b30738ef-7581-4795-a265-dfe84dae8019,Brownies,NaN,GROCERY,CAT_BAKERY_ITEM,562,0.15,CAT_BAKERY_ITEM,562,100,100
304886,31-10-2022 11:31 PM,19-10-2022 5:03 AM,d6ca3c6a-f434-5b73-b4bb-a3d9b2c75429:32973f1e-...,d6ca3c6a-f434-5b73-b4bb-a3d9b2c75429,32973f1e-b047-4501-b0a7-6730a641b5a1,Cupcakes (12 pcs) without Toppings,Per dozen.,GROCERY,CAT_BAKERY_ITEM,562,0.35,CAT_BAKERY_ITEM,562,100,100
304887,31-10-2022 11:31 PM,19-10-2022 5:03 AM,d6ca3c6a-f434-5b73-b4bb-a3d9b2c75429:11558d19-...,d6ca3c6a-f434-5b73-b4bb-a3d9b2c75429,11558d19-3246-4763-a7ee-2b64ba393ad9,Ice Cream (Blue Bell),Sold by the scoop. Per scoop.,GROCERY,"CAT_PREPACKAGED_FOOD,CAT_ICECREAM,TEMP_COLD","106,105,3",0.22,"CAT_PREPACKAGED_FOOD,CAT_ICECREAM,TEMP_COLD","106,105,3",100,100


In [98]:
data_cicd_final3.to_csv('TaxML-CICD - Prod_Data (5).csv')

In [99]:
master_mapping_data = pd.read_csv("../data/MASTER_Tax_Category_Mapping.csv")
set_cicd_categories_aftermapping = set(data_cicd['Agent Corrected CAT Name'].unique())
set_master_mapping_data_categories = set(master_mapping_data['CAT_Name'].unique())
dif_categories = list(set_cicd_categories_aftermapping.difference(set_master_mapping_data_categories))
len(dif_categories)

0

In [14]:

#misclassified data                                        
#data_cicd_misclassification=data_cicd[(data_cicd['CAT NAME_ ValidationScore [0-100]']==0) &(data_cicd['Integer_ValidationScore[0-100]']==0)]
data_cicd_latest=data_cicd[['UniqueUUID','Item','Description','establishment_type','Agent Corrected CAT Name standarized', 'Agent Corrected Integer standardized']]
data_cicd_latest['target_new']=data_cicd_latest['Agent Corrected CAT Name standarized'] + ":" + data_cicd_latest['Agent Corrected Integer standardized']
data_cicd_latest.drop(['Agent Corrected CAT Name standarized', 'Agent Corrected Integer standardized'],inplace=True,axis=1)
print(data_cicd_latest.shape)

(539198, 5)


/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [102]:
#read input from historical data into dataframe
data_historical = pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/historical_data_24_11_22.csv', encoding='utf8',engine='python',usecols=['UniqueUUID','Item','Description','establishment_type','target_new'])

# data analysis and check

In [103]:
data_historical.shape

(396190, 5)

In [104]:
data_historical.columns

Index(['UniqueUUID', 'Item', 'Description', 'establishment_type',
       'target_new'],
      dtype='object')

In [105]:
data_historical.target_new.nunique()

204

In [106]:
data_historical1 = data_historical
data_historical1[['Agent_cat_name','Agent_int']]= data_historical1.target_new.str.split(":",expand=True)

In [107]:
data_historical1['Agent_cat_name'].nunique()

204

In [108]:
set_historical_categories = set(data_historical1['Agent_cat_name'].unique())

In [109]:
set_historical_categories

{'CAT_ALCOHOL,TEMP_COLD',
 'CAT_ANTI_FREEZE',
 'CAT_BABY_FORMULA',
 'CAT_BABY_WIPES',
 'CAT_BAKERY_ITEM',
 'CAT_BAKERY_ITEM_GROCERY_STORE',
 'CAT_BANDAGES',
 'CAT_BATHING_SUITS',
 'CAT_BATTERIES',
 'CAT_BEER',
 'CAT_BREATH_MINTS',
 'CAT_CANDY,TEMP_COLD',
 'CAT_CANDY,TRAIT_FLOUR,TEMP_COLD',
 'CAT_CANDY_COATED_NUTS',
 'CAT_CHARCOAL_BRIQUETTES',
 'CAT_CHOCOLATE',
 'CAT_CIDER',
 'CAT_CLOTHING',
 'CAT_COFFEE,CONTAINER_BOTTLED,TEMP_COLD',
 'CAT_COMP_HARDWARE',
 'CAT_CONDOMS',
 'CAT_CONFECTIONARY',
 'CAT_CONTACT_LENS_SOLUTION',
 'CAT_COSTUMES',
 'CAT_DELI_PLATTER,TEMP_UNHEATED',
 'CAT_DIAPERS',
 'CAT_DISPOSABLE_GLOVES',
 'CAT_ENERGY_DRINK,CONTAINER_BOTTLED,TEMP_COLD',
 'CAT_ENGINE_OIL',
 'CAT_FEMININE_HYGIENE_PRODUCTS',
 'CAT_FERTILIZER',
 'CAT_FIREWOOD',
 'CAT_FIRST_AID_KITS',
 'CAT_FLASHLIGHT',
 'CAT_FOOD_BY_WT_VOL,TEMP_UNHEATED',
 'CAT_FOOTWEAR',
 'CAT_FORTIFIED_WINE',
 'CAT_FRUIT_VEG_PLANTS',
 'CAT_GIFT_CARDS',
 'CAT_GLOVES',
 'CAT_GUM',
 'CAT_HAND_SANITIZER',
 'CAT_ICE,TEMP_COLD',
 'CAT_

In [110]:
#set_master_mapping_data_categories
dif_categories = list(set_historical_categories.difference(set_master_mapping_data_categories))
len(dif_categories)

0

In [106]:
#dif_categories,data_historical.shape,data_historical.columns

[]

In [25]:
data_cicd_latest.columns

Index(['UniqueUUID', 'Item', 'Description', 'establishment_type',
       'target_new'],
      dtype='object')

In [26]:
data_historical = data_historical.drop(['Agent_cat_name','Agent_int'], axis=1)

In [27]:
data_historical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396190 entries, 0 to 396189
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   UniqueUUID          396190 non-null  object
 1   Item                396187 non-null  object
 2   Description         271012 non-null  object
 3   establishment_type  396190 non-null  object
 4   target_new          396190 non-null  object
dtypes: object(5)
memory usage: 15.1+ MB


In [28]:
data_cicd_latest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 539198 entries, 0 to 539846
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   UniqueUUID          539198 non-null  object
 1   Item                539196 non-null  object
 2   Description         316081 non-null  object
 3   establishment_type  539198 non-null  object
 4   target_new          539198 non-null  object
dtypes: object(5)
memory usage: 24.7+ MB


# Merging data_cicd and historical

In [29]:
#mergedStuff_cicd= data_df.set_index('UniqueUUID').join(data_cicd_latest.set_index('UniqueUUID'))
final_data_previous=pd.concat([data_historical, data_cicd_latest], join="outer")

In [30]:
final_data_previous.shape

(935388, 5)

In [31]:
final_data_previous.columns

Index(['UniqueUUID', 'Item', 'Description', 'establishment_type',
       'target_new'],
      dtype='object')

# Check for any unmapped categories

In [29]:
finaldata1 = final_data_previous
finaldata1[['Agent_cat_name','Agent_int']]= finaldata1.target_new.str.split(":",expand=True)

In [30]:
final_data_categories = set(finaldata1['Agent_cat_name'].unique())
dif_categories_final = list(final_data_categories.difference(set_master_mapping_data_categories))
len(dif_categories_final)

0

In [32]:
final_data_previous = final_data_previous.drop(['Agent_cat_name','Agent_int'], axis=1)
final_data_previous.shape

(935388, 5)

# Add latest data

In [33]:
import glob
path = "/Users/mdevar2/Documents/TaxML/retrainingdata/"
csv_files = glob.glob(os.path.join(path, "*.xlsx"))
list_of_dfs = []
  
# loop over the list of csv files
for f in csv_files:
    
    # read the csv file
    df = pd.read_excel(f)
    #print('File Name:', f.split("\\")[-1])
    #df = df[['item_name','description','establishment_type','CAT NAME,CAT_TEMP','Value for Bulk Upload']]
    list_of_dfs.append(df)

In [34]:
final_data1 = pd.concat(list_of_dfs, ignore_index=True)
final_data1.columns

Index(['Unnamed: 0', 'merchant_uuid', 'store_name', 'item_uuid', 'item_name',
       'subsection_name', 'description', 'establishment_type', 'combined_text',
       'processed_text', 'cat_name', 'cat_int', 'prediction_cat_confscore',
       'Tax Category', 'CAT NAME,CAT_TEMP', 'Converted', 'Tagged By', 'Date',
       'Value for Bulk Upload'],
      dtype='object')

In [35]:
#final_data1.to_csv("final_data1.csv")

In [36]:
final_data1.rename(columns = {'item_name':'Item', 'description': 'Description','CAT NAME,CAT_TEMP':'Agent Corrected CAT Name', 'Value for Bulk Upload': 'Agent Corrected Integer'}, inplace = True)
final_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923460 entries, 0 to 923459
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Unnamed: 0                923460 non-null  int64         
 1   merchant_uuid             923460 non-null  object        
 2   store_name                923460 non-null  object        
 3   item_uuid                 923460 non-null  object        
 4   Item                      923460 non-null  object        
 5   subsection_name           923460 non-null  object        
 6   Description               743800 non-null  object        
 7   establishment_type        923460 non-null  object        
 8   combined_text             923460 non-null  object        
 9   processed_text            923460 non-null  object        
 10  cat_name                  923357 non-null  object        
 11  cat_int                   923357 non-null  object        
 12  pr

In [37]:
df5 = pd.read_excel('/Users/mdevar2/Documents/TaxML/retrainingdata_1/INDIRECTTX-2009 - New Products for Tagging - Walgreens - 01.11.2023 - Sheet1_after_tagging.xlsx')
df5.columns

Index(['Unnamed: 0', 'WIC', 'merchant_uuid', 'merchant_name', 'item_uuid',
       'item_name', 'subsection_name', 'description', 'establishment_type',
       'cat_name', 'cat_int', 'prediction_cat_confscore', 'Tax Category',
       'agent_catname', 'agent_int'],
      dtype='object')

In [39]:
df5.rename(columns = {'item_name':'Item', 'description': 'Description','agent_catname':'Agent Corrected CAT Name', 'agent_int': 'Agent Corrected Integer'}, inplace = True)
df5.dtypes

Unnamed: 0                    int64
WIC                           int64
merchant_uuid               float64
merchant_name                object
item_uuid                   float64
Item                         object
subsection_name              object
Description                  object
establishment_type           object
cat_name                     object
cat_int                      object
prediction_cat_confscore    float64
Tax Category                 object
Agent Corrected CAT Name     object
Agent Corrected Integer      object
dtype: object

In [43]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                657 non-null    int64  
 1   WIC                       657 non-null    int64  
 2   merchant_uuid             0 non-null      float64
 3   merchant_name             657 non-null    object 
 4   item_uuid                 0 non-null      float64
 5   Item                      657 non-null    object 
 6   subsection_name           657 non-null    object 
 7   Description               657 non-null    object 
 8   establishment_type        657 non-null    object 
 9   cat_name                  657 non-null    object 
 10  cat_int                   657 non-null    object 
 11  prediction_cat_confscore  657 non-null    float64
 12  Tax Category              657 non-null    object 
 13  Agent Corrected CAT Name  581 non-null    object 
 14  Agent Corr

In [40]:
final_data1 = pd.concat([final_data1, df5], ignore_index=True)
final_data1.columns

Index(['Unnamed: 0', 'merchant_uuid', 'store_name', 'item_uuid', 'Item',
       'subsection_name', 'Description', 'establishment_type', 'combined_text',
       'processed_text', 'cat_name', 'cat_int', 'prediction_cat_confscore',
       'Tax Category', 'Agent Corrected CAT Name', 'Converted', 'Tagged By',
       'Date', 'Agent Corrected Integer', 'WIC', 'merchant_name'],
      dtype='object')

In [33]:
final_data1=pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/retrain_data/combinedlatestdata.csv')

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [34]:
final_data_latest = final_data1[['Item','Description','establishment_type','Agent Corrected CAT Name','Agent Corrected Integer']]
final_data_latest.shape

(924117, 5)

In [35]:
final_data_latest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924117 entries, 0 to 924116
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   Item                      924117 non-null  object
 1   Description               744457 non-null  object
 2   establishment_type        924117 non-null  object
 3   Agent Corrected CAT Name  923780 non-null  object
 4   Agent Corrected Integer   924117 non-null  object
dtypes: object(5)
memory usage: 35.3+ MB


In [36]:
4117 - 3780

337

In [37]:
#final_data_latest.to_csv("final_data_latest_raw.csv")

# Data check for unmapped categories for latest data and map with standard categories

In [38]:
#final_data.dropna(subset=['target_new'],inplace=True)
final_data_latest.dropna(subset=['Agent Corrected CAT Name'],inplace=True)

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [39]:
final_data_latest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 923780 entries, 0 to 924116
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   Item                      923780 non-null  object
 1   Description               744162 non-null  object
 2   establishment_type        923780 non-null  object
 3   Agent Corrected CAT Name  923780 non-null  object
 4   Agent Corrected Integer   923780 non-null  object
dtypes: object(5)
memory usage: 42.3+ MB


In [40]:
#check for unmapped categories
#final_data_latest_categories = set(final_data_latest['Agent Corrected CAT Name'].unique())
##len(dif_categories_final_data_latest)

NameError: name 'set_master_mapping_data_categories' is not defined

# Map latest data with standard categories

In [45]:
#mapping to the standard categories and integers
final_data_latest["Agent Corrected CAT Name standarized"] = final_data_latest["Agent Corrected CAT Name"].replace(cat_dict)

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [46]:
final_data_latest['Agent Corrected Integer standardized'] = final_data_latest['Agent Corrected CAT Name standarized'].map(mapping1)

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
final_data_latest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 923780 entries, 0 to 924116
Data columns (total 7 columns):
 #   Column                                Non-Null Count   Dtype 
---  ------                                --------------   ----- 
 0   Item                                  923780 non-null  object
 1   Description                           744162 non-null  object
 2   establishment_type                    923780 non-null  object
 3   Agent Corrected CAT Name              923780 non-null  object
 4   Agent Corrected Integer               923780 non-null  object
 5   Agent Corrected CAT Name standarized  923780 non-null  object
 6   Agent Corrected Integer standardized  923780 non-null  object
dtypes: object(7)
memory usage: 56.4+ MB


In [48]:
final_data_latest.head()

,Item,Description,establishment_type,Agent Corrected CAT Name,Agent Corrected Integer,Agent Corrected CAT Name standarized,Agent Corrected Integer standardized
0,Chunky Blue Cheese Dressing 16 fl oz,Kraft Chunky Blue Cheese Dressing,GROCERY,CAT_PREPACKAGED_FOOD_CONDIMENTS,740,CAT_PREPACKAGED_FOOD_CONDIMENTS,740
1,Wonderful Pistachios No Shells Sea Salt & Vine...,"Pistachios, Sea Salt & Vinegar, No Shells, Bag",CONVENIENCE,CAT_PREPACKAGED_FOOD_SNACK_NUTS,747,CAT_PREPACKAGED_FOOD_SNACK_NUTS,747
2,Tylenol Sinus Congestion & Pain 24 count,Tylenol SinusÂ Severe Daytime Caplets offer te...,CONVENIENCE,CAT_OTC_MEDICATION,521,CAT_OTC_MEDICATION,521
3,Off Deep Woods Aerosol 6oz,OFF!Â Deep WoodsÂ®Â Insect Repellent V provide...,CONVENIENCE,CAT_TPP_INSECT_REPELLENT,805,CAT_TPP_INSECT_REPELLENT,805
4,Blood Orange Lime Juice with CBD 14 fl oz,Hemp CBD 25 mg,GROCERY,CAT_SUPPLEMENTS,542,CAT_SUPPLEMENTS,542


In [49]:
#creating target_new variable
final_data_latest['target_new']=final_data_latest['Agent Corrected CAT Name standarized'] + ":" + final_data_latest['Agent Corrected Integer standardized']
final_data_latest.drop(['Agent Corrected CAT Name','Agent Corrected Integer','Agent Corrected CAT Name standarized', 'Agent Corrected Integer standardized'],inplace=True,axis=1)
print(final_data_latest.shape)

(923780, 4)


/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [50]:
#final_data_latest = final_data_latest[['Item','Description','establishment_type','target_new']]
#print(final_data_latest.shape)

In [52]:
#cross checking if all categories are mapped
final_data_latest_categories1 = set(final_data_latest['Agent Corrected CAT Name standarized'].unique())
dif_categories_final_data_latest1 = list(final_data_latest_categories1.difference(set_master_mapping_data_categories))
len(dif_categories_final_data_latest1)

0

In [224]:
dif_categories_final_data_latest

[]

In [262]:
#final_data_latest.to_csv("final_data_latest_aftermapping.csv")

In [226]:
final_data_latest.shape

(923780, 4)

In [227]:
#final_data_latest.to_csv("final_data_latest4.csv")

# Combining the previous data with latest data

In [51]:
final_data_previous.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 935388 entries, 0 to 539846
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   UniqueUUID          935388 non-null  object
 1   Item                935383 non-null  object
 2   Description         587093 non-null  object
 3   establishment_type  935388 non-null  object
 4   target_new          935388 non-null  object
dtypes: object(5)
memory usage: 42.8+ MB


In [52]:
final_data_previous1 = final_data_previous[['Item', 'Description', 'establishment_type', 'target_new']]

In [53]:
final_data_combined = pd.concat([final_data_previous1, final_data_latest],ignore_index=True)
final_data_combined.shape

(1859168, 4)

In [64]:
final_data_combined = final_data_combined.astype(str)

In [65]:
final_data_combined.dtypes

Item                  object
Description           object
establishment_type    object
target_new            object
dtype: object

# preprocessing

In [55]:
final_data_combined['combined_text'] = final_data_combined[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)

In [57]:
#final_data_combined = pd.concat([final_data_previous[['Item', 'Description', 'establishment_type', 'target_new']], final_data_latest])
# combine the columns Item, Description and establishment_type into one column 'combined_text'
#final_data_combined['combined_text'] = final_data_combined[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
final_data_combined['processed_text']= final_data_combined['combined_text'].map(lambda s:preprocess_text(s)) 
print(final_data_combined.shape)

(1859168, 6)


In [58]:
final_data_combined=final_data_combined.drop_duplicates(subset=['processed_text','target_new'],keep='first')
print(final_data_combined.shape)
#remove rows having empty target column
final_data_combined.dropna(subset=['target_new'],inplace=True)

X= final_data_combined[['Item','Description','establishment_type','processed_text']]
y= final_data_combined['target_new']

# split the cicd data into train and test 
X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=True, test_size = .20, random_state = 42)


(573196, 6)


/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [254]:
#final_data_combined.to_csv("final_data_combined_afterpreprocessing.csv")